In [1]:
import pandas as pd
import json
import os
import numpy as np
from pprint import pprint
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [2]:
os.environ['SPOTIPY_CLIENT_ID'] = "99b3295c313d45088cb4f8fe0f8dbb81"
os.environ['SPOTIPY_CLIENT_SECRET'] = "ac4fa266a80d43e3beb6b31d2b88dcfb"
os.environ['OAUTH_TOKEN_URL'] = 'https://accounts.spotify.com/api/token'
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials())

In [3]:
def build_API_data():
    file = open(os.path.join('..','Outputs', 'cleaned_data.json'),"r")
    data = json.load(file)
    df = pd.json_normalize(data)
    return df, data

In [4]:
df, file_data = build_API_data()

In [5]:
df

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0DuWDLjriRPjDRoPgaCslY,Adele,25,Love In The Dark,78,285935,False,0.331,0.341,9,-6.057,0,0.0309,0.52800,0.000000,0.1090,0.152,109.821,4,british
1,3di5hcvxxciiqwMH1jarhY,Adele,21,Set Fire to the Rain,80,242973,False,0.603,0.670,2,-3.882,0,0.0249,0.00408,0.000002,0.1120,0.446,107.993,4,british
2,6DEMMeWXfmFAXgDUMMzeg6,Adele,25,All I Ask,74,271800,False,0.591,0.280,4,-5.494,1,0.0283,0.88900,0.000000,0.1240,0.348,141.916,4,british
3,0gplL1WMoJ6iYaPgMCL0gX,Adele,Easy On Me,Easy On Me,85,224694,False,0.604,0.366,5,-7.519,1,0.0282,0.57800,0.000000,0.1330,0.130,141.981,4,british
4,3bNv3VuUOKgrf5hu3YcuRo,Adele,21,Someone Like You,81,285240,False,0.556,0.319,9,-8.251,1,0.0281,0.89300,0.000000,0.0996,0.294,135.187,4,british
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8934,4DqNBM2oUr5R87cUd2E3P6,Josh A,LONELY VIBES (Deluxe),PTSD,41,121867,False,0.644,0.637,7,-7.922,1,0.0898,0.38200,0.000000,0.1410,0.216,81.479,4,sad
8935,6HdrRVMieDZtv0vldfZxqg,Rarin,Day Ones,Day Ones,47,134913,True,0.739,0.639,9,-8.636,1,0.2530,0.11900,0.000000,0.1190,0.482,143.044,4,sad
8936,6hzTeCA93yeY8WZE0kW3ui,Powfu,some boring love stories pt 5,clash of the titans,42,110000,False,0.727,0.419,1,-8.334,1,0.1870,0.59500,0.000000,0.1120,0.751,158.344,4,sad
8937,7sFRIgzKlKRSfBDWUvtpwU,LoveJSan,Saddest Popstar to Live,I Got You Flowers!,49,154000,True,0.598,0.540,7,-6.199,1,0.0363,0.05040,0.000144,0.2110,0.244,140.130,4,sad


In [6]:
chunks = [file_data[i:i + 50] for i in range(0, len(file_data), 50)]

In [7]:
chunked_ids = [[track['track_id'] for track in chunk] for chunk in chunks]
print(len(chunked_ids[-1]))

39


In [8]:
# %time
# test_chunk = chunked_ids[-2]

In [9]:
# for track in test_search['tracks']:
#     print({'track_id': track['id'],'name': track['name'], 'url': track['external_urls']['spotify'], 'popularity': track['popularity']})

In [10]:
# %time
# file_data = [dict(track, **{'track_url': spotify.track(track['track_id'])['external_urls']['spotify']}) for track in file_data]

In [ ]:
%%time
not_found = 0
for index, track in df.iterrows():
    try:
        track_name = track['track_name']
        track_id = track['track_id']
        artist = track['artists']
        album = track['album_name']
        result = spotify.search(f'track:{track_name} artist:{artist} album:{album}', type='track')
        # pprint(result)
        url = result['tracks']['items'][0]['external_urls']['spotify']
        # print(url)
        df.loc[index,'track_url'] = url
    except:
        not_found += 1
        df.loc[index,'track_url'] = None
    

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': 'track:Maybe It’s Time (feat. Corey Taylor, Joe Elliott, Brantley Gilbert, Ivan Moody, Slash, AWOLNATION, Tommy Vext) artist:Sixx:A.M.;Corey Taylor;Joe Elliott;Brantley Gilbert;Ivan Moody;Slash;AWOLNATION;Tommy Vext album:Maybe It’s Time (feat. Corey Taylor, Joe Elliott, Brantley Gilbert, Ivan Moody, Slash, AWOLNATION, Tommy Vext)', 'limit': 10, 'offset': 0, 'type': 'track', 'market': None} returned 400 due to Bad request.


In [ ]:
display(df[['track_name', 'track_url']])

In [ ]:
print(f"{not_found} urls not found')